In [1]:
# 把網頁資料爬好存到資料庫
# 程式 10-1.py (Python 3 version)

import sqlite3
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as pt

conn = sqlite3.connect('Test_data.sqlite')


#選單
def disp_menu():
    print("租屋查詢系統")
    print("------------")
    print("1.從網站載入最新屋況")
    print("2.顯示歷年油價資訊")
    print("3.最近10週油價資訊")
    print("4.油價走勢圖")
    print("0.結束")
    print("------------")


#新增(爬)資料
def fetch_data():
    url = 'https://www.rakuya.com.tw/search/rent_search/index?con=eJyrVkrOLKlUsopWMlCK1VFKySwuyEkE8pVyMotLlHSUslMry_OLUkAKYkpNTYyNY0otjAwsgKSFqTlIR35SVmYeWD4IxC1OTSxKzgBxwebG1gIAtr0d2g&upd=1'
    
    html = requests.get(url).text
    sp = BeautifulSoup(html, 'html.parser')
    data = sp.find_all('div', {'class':'content type-list clearfix'})
    rows = data[0].find_all('div':'class':'obj-title')

    prices = list()
    for row in rows:
        cols = row.find_all('li': {'class':'obj-price'})
        if len(cols[1].text) > 0:
            item = [cols[0].text, cols[1].text, \
                    cols[2].text, cols[3].text]
            prices.append(item)
    for p in prices:
        sqlstr = "select * from prices where gdate='{}';".format(p[0])
        cursor = conn.execute(sqlstr)
        if len(cursor.fetchall()) == 0:
            g92 = 0 if p[1]=='' else float(p[1])
            g95 = 0 if p[2]=='' else float(p[2])
            g98 = 0 if p[3]=='' else float(p[3])
            sqlstr = "insert into prices values('{}', {}, {}, {});". \
                format(p[0], g92, g95, g98)
            print(sqlstr)
            conn.execute(sqlstr)
            conn.commit()

#展示
def disp_10data():
    cursor = conn.execute('select * from prices order by gdate desc;')
    n = 0
    for row in cursor:
        print("日期：{}，92無鉛：{}，95無鉛：{}，98無鉛：{}". \
            format(row[0],row[1],row[2],row[3]))
        n = n + 1
        if n == 10:
            break